The following work will implement the following:
1) given a stock ticker, we will retrieve latest news of the company (news agent)
2) given the news, the sentiment agent will evaluate the linked sentiment to the news
3) given the sentiment, a final agent will emit a recommendation to buy, sell or hold the ticker.

Discussion with claude at
https://claude.ai/chat/4a1f5da0-e13e-441d-9597-af8626152811

In [1]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate # this is usedas a base class to create custom prompts
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

In [2]:
from typing import List, Union
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
type(os.environ["OPENAI_API_KEY"])

str

# News agent

In [5]:
search = DuckDuckGoSearchRun()

In [6]:
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="this will search for stock ticker news"
    )
]

Now we create the template to supply a custom prompt to the LLM.

In [13]:
class CustomPromptTemplate(StringPromptTemplate):
    """
    This object allows us to feed into the LLM the custom prompt into the agent
    """
    template: str
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        """
        This function formats the given prompt using
            intermediate steps
            thoughts
            actions
        as defined
        """
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""

        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "

        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])

        return self.template.format(**kwargs)
    
    def _get_input_variables(self):
        """
        Returns input variables required by prompt template
        """
        return ["input", "intermediate_steps", "tools", "tool_names"]

In [15]:
prompt = CustomPromptTemplate(
    template="""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}""",
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [16]:
# Define the LLM
llm = OpenAI(temperature=0)

c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


ImportError: Could not import openai python package. Please install it with `pip install openai`.

In [ ]:
from typing import List, Union
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv